# Pump systems

A pump is used to move fluid through a pipeline.

<img src=pump2.png width=300 align=center>

* The flow rate is controlled by a valve.
* The pump supplies pressure at various flow rates. 
* If we divide the pressure by the fluid density and gravity, which are both constant, we get units of length, which we call a **head**.
* So, we can plot **head** versus flow rate. Here's an example of a ***pump curve***
    * We'll consider the 12.75 inch pump size (top curve).

<img src=pump.png width=600 align=center>

* The flow through the system is related by a mechanical energy balance:

$$
\frac{\Delta P}{\rho g} + \frac{\Delta v^2}{2g} + \Delta z = h_p -\frac{fLv^2}{2Dg} - \frac{Kv^2}{2g}
$$
* Here, $z$ is height, $f$ is the pipe friction factor, $L$ is the pipe length, $D$ is the pipe diameter, and $K$ is a valve coeficient.
* We'll assume the tanks are open to the atmosphere, have no vecocity at the top surface, and are at the same height, so that the left hand side is zero. We're assuming no other friction losses besides the pipe and the valve. Rearrange to
<font color=blue>
    
$$
h_p = \frac{fLv^2}{2Dg} + \frac{Kv^2}{2g}.
$$
</font>

* The system operates where the **operating curve (equation above)** and the **pump curve** in the figure intersect.
    * That is, the system can operate anywhere on the operating curve defined by the equation above, (that is, any head/flow combination given by that equation), **but because the flow is driven by a pump, we are restricted to the points that also lie on the pump curve.**
    * We adjust the intersection point, that is, the operating point of the system, by opening or closing the valve (changing $K$). This will effectively pivot the operating curve left or right about the origin.

Consider a pump system with a pipe with the following parameters:
* L = 35 m
* f = 0.015
* D = 0.1

The operating point can be visualizaed by opening/closing a valve (changing $K$).


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
import pint; u = pint.UnitRegistry()

import ipywidgets as wg
from IPython.display import display

Qp = np.array([1.16, 40.8, 80.5, 121, 161, 201, 240, 281, 321, 359, 401, 440, 480, 519, 560, 600, 640, 680, 720, 760, 800, 827])*u.gal/u.min
hp = np.array([76.2, 76.2, 76.3, 76.0, 76.1, 75.6, 75.2, 74.7, 74.1, 73.0, 71.3, 69.7, 68.0, 65.7, 63.3, 60.5, 57.3, 53.7, 49.1, 43.7, 36.7, 30.9])*u.ft
Qp.ito(u.m**3/u.s)
hp.ito(u.m)
Qp = Qp.magnitude
hp = hp.magnitude

def operatingcurve(Q, K):
    L = 35
    D = 0.1
    f = 0.015
    g = 9.81
    
    v = 4*Q/np.pi/D**2
    
    return f*L*v**2/2/D/g + K*v**2/2/g

pumpcurve = interp1d(Qp, hp, fill_value='extrapolate')

def get_operating_point(K):
    
    def F(Q,K):
        return operatingcurve(Q,K)-pumpcurve(Q)
    
    Q = fsolve(F, 0.02, args=(K,))[0]
    h = operatingcurve(Q,K)
    print(f"Q={Q:.3f} m^3/s,\nh={h:.1f} m")
    
    Qpts = np.linspace(np.min(Qp), np.max(Qp), 100)
    plt.plot(Qpts, operatingcurve(Qpts,K))
    plt.plot(Qpts, pumpcurve(Qpts))
    plt.plot([Q], [h], 'o', markersize=10)
    plt.ylim([0,30])
    plt.xlabel(r'Q (m$^3$/s)')
    plt.ylabel('h (m)')
    

wg.interact(get_operating_point, K=(0,100,5));

interactive(children=(IntSlider(value=50, description='K', step=5), Output()), _dom_classes=('widget-interact'…